<a href="https://colab.research.google.com/github/NikolayLenkovNikolaev/R_for_Pharmacometrics/blob/main/Dose_PK_ExposureSingle_Ascxending_Dose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [472]:
def trapez(df):
  x = df["TIME"].to_list()
  y = df["LIDV"].to_list()
  area = np.trapz(y, x)

  return area

## Overview

This document contains exploratory plots for single ascending dose PK data as well as the Python code that generates these graphs. The plots presened here are based on simulated data form Novartis.


## Setup

In [473]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

from packaging import version
import sklearn
print(f"Sciket Learn version:   {sklearn.__version__}")
print(f"Lifelines version:      {sklearn.__version__}")
print(f"Sciket Survive version: {sklearn.__version__}")

from google.colab import drive
drive.mount('/content/drive/')

Sciket Learn version:   1.2.2
Lifelines version:      1.2.2
Sciket Survive version: 1.2.2
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Load Dataset

In [474]:
# load dataset
path = "/content/drive/MyDrive/Python_pharma/data/"

file = "Single_Ascending_Dose_Dataset2.csv"

pkpd_data = pd.read_csv(path+file)
pkpd_data.head(3)

,ID,TIME,NOMTIME,TIMEUNIT,AMT,LIDV,CMT,NAME,EVENTU,CENS,EVID,WEIGHTB,SEX,TRTACT,DOSE
0,1,-0.089,-0.1,Hours,0,NaN,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100
1,1,0.000,0.0,Hours,100,NaN,1,Dosing,mg,0,1,77.9,Male,100 mg,100
2,1,0.170,0.1,Hours,0,0.665,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100


In [475]:
#ensure dataset has all the necessary columns
pkpd_data["LIDV_NORM"] = pkpd_data["LIDV"] / pkpd_data["DOSE"]
pkpd_data["LIDV_UNIT"] = np.where(pkpd_data["CMT"] == 2, "ng/ml", "")

In [476]:
pkpd_data.head()

,ID,TIME,NOMTIME,TIMEUNIT,AMT,LIDV,CMT,NAME,EVENTU,CENS,EVID,WEIGHTB,SEX,TRTACT,DOSE,LIDV_NORM,LIDV_UNIT
0,1,-0.089,-0.1,Hours,0,NaN,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100,NaN,ng/ml
1,1,0.000,0.0,Hours,100,NaN,1,Dosing,mg,0,1,77.9,Male,100 mg,100,NaN,
2,1,0.170,0.1,Hours,0,0.665,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100,0.00665,ng/ml
3,1,0.565,0.5,Hours,0,0.997,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100,0.00997,ng/ml
4,1,1.145,1.0,Hours,0,1.350,2,PK Concentration,ng/mL,0,0,77.9,Male,100 mg,100,0.01350,ng/ml


In [477]:
# FILTER

pkpd_data = pkpd_data[pkpd_data["ID"].isin([1,16])]
pkpd_data.shape

(28, 17)

In [478]:
pkpd_data = pkpd_data.sort_values("DOSE")
pkpd_data["TRTACT_low2high"] = pkpd_data["TRTACT"]
pkpd_data["TRTACT_high2low"] = pkpd_data["TRTACT"]
pkpd_data = pkpd_data[['ID', 'TIME', 'NOMTIME', 'TIMEUNIT', 'AMT', 'LIDV', 'CMT', 'NAME',
                        'EVENTU', 'CENS', 'EVID', 'WEIGHTB', 'SEX', 'DOSE',
                         'LIDV_NORM', 'LIDV_UNIT', 'TRTACT_low2high', 'TRTACT_high2low']]

In [479]:
pk_data = pkpd_data[pkpd_data["CMT"] == 2]
pk_data.shape

(26, 18)

In [480]:
pk_data.groupby(["ID", "DOSE"])
NCA = pk_data[pk_data['LIDV'].notna()]
NCA.shape

(24, 18)

In [481]:
#NCA= NCA[["ID", "TIME", "LIDV"]]
NCA = NCA.sort_values(["ID", "TIME"])
NCA.shape

df = pd.DataFrame()
df["AUC_last"] = NCA.groupby("ID").apply(trapez)

(24, 18)

In [482]:
def get_max(df):
  max = df["LIDV"].max()
  return max

df["C_max"] = NCA.groupby("ID").apply(get_max)
df

,AUC_last,C_max
ID,,
1,10.984329,1.35
16,40.796805,3.64


In [483]:
def get_first(df, str):
  L = df[str].to_list()
  U = L[0]
  return U
df["SEX"]    = NCA.groupby("ID").apply(get_first, "SEX")
df["WEIGHTB"] = NCA.groupby("ID").apply(get_first, "WEIGHTB")
df["DOSE"] = NCA.groupby("ID").apply(get_first, "DOSE")
df["ID"] = NCA.groupby("ID").apply(get_first, "ID")
df = df.reset_index(drop=True)

df = df[["ID", "DOSE", "AUC_last", "C_max", "SEX", "WEIGHTB"]]

df
df2 = df[["ID", "DOSE", "SEX", "WEIGHTB"]]

df1 = df[["ID", "AUC_last", "C_max"]]
df1
df2

,ID,DOSE,AUC_last,C_max,SEX,WEIGHTB
0,1,100,10.984329,1.35,Male,77.9
1,16,200,40.796805,3.64,Female,93.3


,ID,AUC_last,C_max
0,1,10.984329,1.35
1,16,40.796805,3.64


,ID,DOSE,SEX,WEIGHTB
0,1,100,Male,77.9
1,16,200,Female,93.3


In [484]:
df1 = pd.melt(df1, id_vars=['ID'],
        value_vars=['AUC_last', 'C_max'], var_name='PARAM', value_name='VALUE',  ignore_index=False)

In [485]:
df1

,ID,PARAM,VALUE
0,1,AUC_last,10.984329
1,16,AUC_last,40.796805
0,1,C_max,1.350000
1,16,C_max,3.640000


In [486]:
df3 = df1.merge(df2, left_on='ID', right_on='ID')

In [487]:
df3["VALUE_NORM"] = df3["VALUE"]/df3["DOSE"]
df3

,ID,PARAM,VALUE,DOSE,SEX,WEIGHTB,VALUE_NORM
0,1,AUC_last,10.984329,100,Male,77.9,0.109843
1,1,C_max,1.350000,100,Male,77.9,0.013500
2,16,AUC_last,40.796805,200,Female,93.3,0.203984
3,16,C_max,3.640000,200,Female,93.3,0.018200
